In [1]:
# AusCycling Optimization Problem
# Date: May 2, 2024

# (DONE) Translate bends into laps
# (DONE) Print individual cyclist total_work_depleted percentage
# (DONE) Print total cyclist work depleted as a percentage
# (DONE) translate 6.95 seconds per 125 m to km/hr
# (DONE) Starting and ending cases
# (DONE) Double check negative power corner case
# (DONE) Constraint of P >= pCdAV^3
#      Ask Julian: why 4 CdA values?
#      Ask Julian: where can I find p?
#      Is what unit is V in?
# (DONE) For loop for number of intervals
# (DONE) Code clean up: read csv file in, convert to matrices
# (DONE) Create results dictionary
# (DONE) For loop for cyclists order
#     use: for permutation in permutations(my_list):
# (DONE) For loop for velocities
# (DONE) Acceleration
# (DONE) Add model infeasible catch

In [1]:
# !pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from itertools import permutations
import csv

In [2]:
# Unchanging variables
num_cyclists = 4
num_bends = 32
drags = [.96, .58, .52, .53]

time = [1,5,10,20,30,45,60,120,150,180,240,300,360,480,600,720,900,1200,2400,3600]

half_lap_distance = 125
penalty_distance = 2.1
rho = 1.225
cyclists_init_order = [0,1,2,3]

critical_power_from_csv = []
w_prime_from_csv = []
CdA_from_csv = []
m_from_csv = []
with open('physiology.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row_number, row in enumerate(reader):
        if row_number >= 4:
            break
        critical_power_from_csv.append(float(row['CP']))
        w_prime_from_csv.append(float(row["W'"])*1000)
        CdA_from_csv.append(float(row['CdA']))
        m_from_csv.append(float(row['m']))
print(critical_power_from_csv)
print(w_prime_from_csv)
print(CdA_from_csv)
team_w_prime = sum(w_prime_from_csv)
raw_power_curves_from_csv = []
with open('power_curve.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    m1 = []
    m2 = []
    m3 = []
    m4 = []
    for row in reader:
        m1.append(float(row['M1']))
        m2.append(float(row['M2']))
        m3.append(float(row['M3']))
        m4.append(float(row['M4']))
raw_power_curves_from_csv.append(m1)
raw_power_curves_from_csv.append(m2)
raw_power_curves_from_csv.append(m3)
raw_power_curves_from_csv.append(m4)
print(raw_power_curves_from_csv)

[420.0, 388.0, 420.0, 350.0]
[29100.0, 28600.0, 25500.0, 26100.0]
[0.1978, 0.195, 0.1793, 0.17]
[[1607.0, 1520.0, 1440.0, 1300.0, 1181.0, 1037.0, 926.0, 680.0, 624.0, 591.0, 541.0, 517.0, 501.0, 481.0, 469.0, 460.0, 452.0, 444.0, 432.0, 412.0], [1632.0, 1540.0, 1456.0, 1308.0, 1183.0, 1031.0, 913.0, 653.0, 595.0, 560.0, 507.0, 483.0, 467.0, 447.0, 436.0, 428.0, 420.0, 412.0, 400.0, 380.0], [1489.0, 1411.0, 1338.0, 1212.0, 1104.0, 974.0, 873.0, 651.0, 601.0, 571.0, 526.0, 505.0, 491.0, 473.0, 463.0, 455.0, 448.0, 441.0, 431.0, 412.0], [1462.0, 1380.0, 1305.0, 1173.0, 1061.0, 926.0, 820.0, 589.0, 537.0, 506.0, 459.0, 437.0, 422.0, 404.0, 393.0, 386.0, 379.0, 372.0, 361.0, 343.0]]


In [3]:
def initialize_W_depleted_matrix(half_lap_time, velocity_m_per_sec, critical_power, w_prime, CdA, raw_power_curves, num_half_laps_to_accel):
    #print(velocity_m_per_sec)
    num_bends_steady_state = num_bends - num_half_laps_to_accel
    max_power = np.zeros((num_cyclists, num_bends_steady_state, num_bends_steady_state))
    W_depleted = np.zeros((num_cyclists, len(drags), num_bends_steady_state, num_bends_steady_state))
    penalty_time = (penalty_distance * half_lap_time) / half_lap_distance
    #print(penalty_time)

    for cyc in range(num_cyclists):
        interpolated_function = interp1d(time, raw_power_curves[cyc], kind = 'cubic')
        for col in range(0, num_bends_steady_state):
            for row in range(col + 1, num_bends_steady_state):
                # Slight modification for first and last lap
                if (col == 0) and (row == num_bends_steady_state - 1):
                    time_in_lead = num_bends_steady_state * half_lap_time + penalty_time
                elif (col == 0) and (row != num_bends_steady_state - 1):
                    time_in_lead = ((row - col) + .5) * half_lap_time + (2 * penalty_time)
                elif (col != 0) and (row != num_bends_steady_state - 1):
                    time_in_lead = (row - col) * half_lap_time + (2 * penalty_time)
                elif (col != 0) and (row == num_bends_steady_state - 1):
                    time_in_lead = ((row - col) + .5) * half_lap_time + penalty_time
                else:
                    print("Missed Case")
            
                max_power[cyc][row][col] = max(interpolated_function(time_in_lead), critical_power[cyc])
                for drag in range(len(drags)):
                    calculated_power = (.5*rho*CdA[cyc]*(velocity_m_per_sec**3)) * drags[drag]
                    if calculated_power <= max_power[cyc][row][col]:
                        power = max(0, calculated_power - critical_power[cyc])
                        work = power * time_in_lead
                        W_depleted[cyc][drag][row][col] = work
                    else:
                        W_depleted[cyc][drag][row][col] = 1000 * w_prime[cyc]
                        #print("Infeasible work: cyc: " + str(cyc) + ", drag: " + str(drag) + ", row: " + str(row) + ", col: " + str(col))
    return(W_depleted)

In [4]:
def print_results(results_dictionary):
# Print results in terms of laps
# The array indexes translate to the following lap equivalents
# [0: 0.25, 1: 0.75, 2: 1.25, 3: 1.75, 4: 2.25, 5: 2.75, 6: 3.25, 7: 3.75, 8: 4.25, 9: 4.75, 10: 5.25, 
#  11: 5.75, 12: 6.25, 13: 6.75, 14: 7.25, 15: 7.75, 16: 8.25, 17: 8.75, 18: 9.25, 19: 9.75, 20: 10.25, 
#  21: 10.75, 22: 11.25, 23: 11.75, 24: 12.25, 25: 12.75, 26: 13.25, 27: 13.75, 28: 14.25, 29: 14.75, 30: 15.25,
#  31: 15.75
    cyclist_order = "Optimal cyclist order is: "
    #print(results_dictionary["cyclist_order"][0])
    for cyclist in range(num_cyclists):
        cyclist_order = cyclist_order + results_dictionary["cyclist_order"][cyclist] + ", "
    cyclist_order = cyclist_order[:-2]
    print(cyclist_order)
    print("")
    
    strategy = str(results_dictionary["number_of_intervals"]) + " intervals. Switch leaders at laps: " + str(results_dictionary["switch_strategy"])
    n = results_dictionary["number_of_intervals"]
    print(strategy)
    print("")
    print("Team work depletion: " + str(round(results_dictionary["team_work_depletion"], 2)) + " W, " + str(round(results_dictionary["team_work_depletion_percentage"], 2)) + "%")
    for cyclist in range(num_cyclists):
        print("Cyclist " + results_dictionary["cyclist_order"][cyclist] + " depleted " + str(round(results_dictionary["cyclist_work_depletion"][cyclist],2)) + " W, " + str(round(results_dictionary["cyclist_work_depletion_percent"][cyclist], 2)) + "% of their W'")
          
    if (sum(results_dictionary["cyclist_work_depletion"])) == results_dictionary["team_work_depletion"]:
        print("Sum of individual cyclists power equals objective function value")
    else:
        print("Objective Function - sum of individual cyclists work depleted equals " + str(results_dictionary["team_work_depletion"] - sum(results_dictionary["cyclist_work_depletion"])))
    print("")
    print("Expected split time per lap: " + str(results_dictionary["half_lap_time"] * 2) + " (s)")
    print("Expected constant velocity: " + str(round(results_dictionary["velocity_km_per_hour"],2)) + " km/hr")

    

In [5]:
def generate_infeasible_sol_results_dictionary(n, permutation_to_alpha, half_lap_time, velocity_m_per_sec):
    results_dictionary = {}
    results_dictionary["cyclist_order"] = permutation_to_alpha
    results_dictionary["number_of_intervals"] = n
    results_dictionary["switch_strategy"] = {0: -1}
    results_dictionary["team_work_depletion"] = -1
    results_dictionary["team_work_depletion_percentage"] = -1
    results_dictionary["cyclist_work_depletion"] = [-1, -1, -1, -1]
    results_dictionary["cyclist_work_depletion_percent"] = [-1, -1, -1, -1]
    results_dictionary["half_lap_time"] = half_lap_time
    results_dictionary["velocity_km_per_hour"] = velocity_m_per_sec * 3.6
    results_dictionary["feasibility"] = 'infeasible'
    
    return(results_dictionary)


In [6]:
# n is number of switches
def find_optimal_solution(n, permutation_to_alpha, half_lap_time, velocity_m_per_sec, critical_power, w_prime, CdA, m, raw_power_curves, num_half_laps_to_accel, enforce_first_switch_half_lap):
    W_depleted = initialize_W_depleted_matrix(half_lap_time, velocity_m_per_sec, critical_power, w_prime, CdA, raw_power_curves, num_half_laps_to_accel)
    #print(W_depleted[1][1])
    num_bends_steady_state = num_bends - num_half_laps_to_accel
    #print(num_bends_steady_state)
    time_to_accel = (2 * 125 * num_half_laps_to_accel) / velocity_m_per_sec
    results_dictionary = {}

    # Create the model object
    model= gp.Model ("AusCycling_Model")

    # Add the decision variables
    z = model.addVars(num_bends_steady_state, num_bends_steady_state, n, vtype=GRB.BINARY, name="z")

    # Constraints
    ### Maximum work per cyclist constraint (Need to review this constraint)
    for cyclist in range(num_cyclists):
        work_from_acceleration = drags[cyclist] * ((1 / 2) * m[cyclist] * (velocity_m_per_sec**2)) 
        max_work_constraint = model.addConstr(work_from_acceleration + (sum(z[i, j, k] * W_depleted[cyclist, ((cyclists_init_order[cyclist] + (4 - (k % 4))) % 4), i, j] for i in range(num_bends_steady_state) for j in range(num_bends_steady_state) for k in range(n))) <= w_prime[cyclist], name="max_work")

    ### Number of intervals constraint
    number_intervals = model.addConstr(sum(z[i, j, k] for i in range(num_bends_steady_state) for j in range(num_bends_steady_state) for k in range(n)) == n, name="number_intervals")

    ### Number of switches per interval constraint
    for k in range(n):
        number_switches_per_interval = model.addConstr(sum(z[i, j, k] for i in range(num_bends_steady_state) for j in range(num_bends_steady_state)) == 1, name="number_switches_in_interval_" + str(k))
    
    initial_switch = enforce_first_switch_half_lap - num_half_laps_to_accel
    #print(initial_switch)
    first_switch = model.addConstr((z[initial_switch,0,0] == 1), name="enforce switch after acceleration")
    interval_end = model.addConstr((sum(z[num_bends_steady_state-1,j,n-1] for j in range(num_bends_steady_state)) == 1), name="interval_end")

    ### Intervals start and end from an “Active Bend” 
    for s in range(n-1):
        intervals_start_end = model.addConstrs(((sum(z[i, k, s + 1] for i in range(num_bends_steady_state)) == sum(z[k, j, s] for j in range(num_bends_steady_state))) for k in range(num_bends_steady_state)), name="intervals_start_end")

    # Intervals must have bends that go in order
    for col in range(1, num_bends_steady_state):
        in_order_bends_first_row_0 = model.addConstr(z[0, col, 0] == 0, name="in_order_bends")
    for s in range(1,n):
        for col in range(num_bends_steady_state):
            in_order_bends_first_row_other_n = model.addConstr(z[0, col, s] == 0, name="in_order_bends")
    for s in range(n):
        for row in range(1, num_bends_steady_state):
            for col in range(row, num_bends_steady_state):
                in_order_bends = model.addConstr(z[row, col, s] == 0, name="in_order_bends")

    # Objective Function
    total_work_tracker = [0, 0, 0, 0]
    for cyclist in range(num_cyclists):
        work_from_acceleration = drags[cyclist] * ((1 / 2) * m[cyclist] * (velocity_m_per_sec**2)) 
        #work_from_acceleration = 0
        #print("Work from acceleration: " + str(work_from_acceleration))
        total_work_tracker[cyclist] = work_from_acceleration + (sum(z[i, j, k] * W_depleted[cyclist, ((cyclists_init_order[cyclist] + (4 - (k % 4))) % 4), i, j] for i in range(num_bends_steady_state) for j in range(num_bends_steady_state) for k in range(n)))
    total_work = sum(total_work_tracker)            
    objective_function = model.setObjective(total_work, GRB.MAXIMIZE)
    # suppress output
    model.setParam('OutputFlag', 0)

    model.optimize()

    status = model.status
    #print(status)
    if status == gp.GRB.Status.INFEASIBLE:
        print("Model is infeasible, moving on to the next model...")
        results_dictionary = generate_infeasible_sol_results_dictionary(n, permutation_to_alpha, half_lap_time, velocity_m_per_sec)
        # Move on to the next iteration of the loop
        #continue
    else:
        results_dictionary["cyclist_order"] = permutation_to_alpha
        results_dictionary["number_of_intervals"] = n
        results_dictionary["team_work_depletion"] = model.ObjVal
        results_dictionary["team_work_depletion_percentage"] = model.ObjVal / team_w_prime * 100
        results_dictionary["switch_strategy"] = {}
        for k in range(n-1):
            for j in range(num_bends_steady_state):
                for i in range(num_bends_steady_state):
                    # Retrieve the value of the decision variable at indices (i, j, k)
                    value = z[i, j, k].X
                    if value > 0.1:
                        results_dictionary["switch_strategy"][k] = (.5*i)+.25 + (num_half_laps_to_accel/2)
        total_work_output = [0, 0, 0, 0]
        total_work_output_percent = [0, 0, 0, 0]
        z_values = model.getAttr('X', z)

        #z_values[(i, j, k)]
        for cyclist in range(num_cyclists):
            work_from_acceleration = drags[cyclist] * ((1 / 2) * m[cyclist] * (velocity_m_per_sec**2))
            total_work_output[cyclist] = work_from_acceleration + sum(z_values[i, j, k] * W_depleted[cyclist, ((cyclists_init_order[cyclist] + (4 - (k % 4))) % 4), i, j] for i in range(num_bends_steady_state) for j in range(num_bends_steady_state) for k in range(n))
            total_work_output_percent[cyclist] = total_work_output[cyclist] / w_prime[cyclist] * 100
        results_dictionary["cyclist_work_depletion"] = total_work_output
        results_dictionary["cyclist_work_depletion_percent"] = total_work_output_percent
        results_dictionary["half_lap_time"] = half_lap_time
        results_dictionary["velocity_km_per_hour"] = velocity_km_per_hour
        results_dictionary["feasibility"] = 'feasible'

    return(results_dictionary)

In [7]:
# Variables that may change
csv_file_path = '5_8_24_data3.csv'
init_num_intervals = 7
num_intervals_to_try = 5
half_lap_times = [6.50, 6.55, 6.60, 6.65, 6.70]
#half_lap_times = [6.7]
num_half_laps_to_accel = 4
enforce_first_switch_lap = 4
#perms = [(0,1,2,3)]

final_num_intervals = init_num_intervals + num_intervals_to_try
optimal_team_work_depletion = 0
optimal_number_of_intervals = 0
optimal_strategy = generate_infeasible_sol_results_dictionary(init_num_intervals, ('A','B','C','D'), half_lap_times[0], 125/half_lap_times[0])

#FINISH
with open(csv_file_path, mode='w', newline='') as file:
    # find optimal cyclist order
    writer = csv.writer(file)
    column_titles = ['Permutation', 'Num_Intervals', 'Strategy', 'Team Work Depletion', 'Team Work Depletion %', 
                     'Cyclist init pos 0 Work Depleted', 'Cyclist init pos 0 Work Depleted %',
                     'Cyclist init pos 1 Work Depleted', 'Cyclist init pos 1 Work Depleted %',
                     'Cyclist init pos 2 Work Depleted', 'Cyclist init pos 2 Work Depleted %',
                     'Cyclist init pos 3 Work Depleted', 'Cyclist init pos 3 Work Depleted %',
                     'Expected Split Time per Lap', 'Expected Constant Velocity', 'Feasibility']
    writer.writerow(column_titles)
    for half_lap_time in half_lap_times:
        velocity_m_per_sec = half_lap_distance / half_lap_time
        velocity_km_per_hour = velocity_m_per_sec * 3.6
        for permutation in permutations(cyclists_init_order):
        #for permutation in perms:
            critical_power = [0, 0, 0, 0]
            w_prime = [0, 0, 0, 0]
            CdA = [0, 0, 0, 0]
            m = [0, 0, 0, 0]
            raw_power_curves = [[],[],[],[]]
            for i in range(num_cyclists):
                critical_power[i] = critical_power_from_csv[permutation[i]]
                w_prime[i] = w_prime_from_csv[permutation[i]]
                CdA[i] = CdA_from_csv[permutation[i]]
                m[i] = m_from_csv[permutation[i]]
                raw_power_curves[i] = raw_power_curves_from_csv[permutation[i]]    
   
            # find optimal number of intervals
            for num_intervals in range(init_num_intervals, final_num_intervals):
                permutation_to_alpha = (chr(permutation[0] + 65), chr(permutation[1] + 65), chr(permutation[2] + 65), chr(permutation[3] + 65))
                results_dictionary = find_optimal_solution(num_intervals, permutation_to_alpha, half_lap_time, velocity_m_per_sec, critical_power, w_prime, CdA, m, raw_power_curves, num_half_laps_to_accel, enforce_first_switch_lap)
                print("Testing permutation " + str(permutation_to_alpha) + " at " + str(num_intervals) + " intervals at " + str(half_lap_time) + " (s) half lap time.")
                print("Team work depletion for " + str(num_intervals) + " intervals: " + str(results_dictionary["team_work_depletion"]))
                strategy_dict = results_dictionary["switch_strategy"]
                strategy = tuple(strategy_dict.values())
                csv_data = [permutation_to_alpha, num_intervals, strategy, results_dictionary["team_work_depletion"], results_dictionary["team_work_depletion_percentage"],
                            results_dictionary["cyclist_work_depletion"][0], results_dictionary["cyclist_work_depletion_percent"][0],
                            results_dictionary["cyclist_work_depletion"][1], results_dictionary["cyclist_work_depletion_percent"][1],
                            results_dictionary["cyclist_work_depletion"][2], results_dictionary["cyclist_work_depletion_percent"][2],
                            results_dictionary["cyclist_work_depletion"][3], results_dictionary["cyclist_work_depletion_percent"][3],
                            results_dictionary["half_lap_time"]* 2, results_dictionary["velocity_km_per_hour"], results_dictionary["feasibility"]]
                writer.writerow(csv_data)
                if results_dictionary["team_work_depletion"] > optimal_team_work_depletion:
                    optimal_team_work_depletion = results_dictionary["team_work_depletion"]
                    optimal_number_of_intervals = num_intervals
                    optimal_strategy = results_dictionary
print("")        
print_results(optimal_strategy)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-15
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'B', 'C', 'D') at 7 intervals at 6.5 (s) half lap time.
Team work depletion for 7 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'B', 'C', 'D') at 8 intervals at 6.5 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'B', 'C', 'D') at 9 intervals at 6.5 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'B', 'C', 'D') at 10 intervals at 6.5 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'B', 'C', 'D') at 11 intervals at 6.5 (s) half lap time.
Team work depletion for 11 intervals: -1
Model is infeasible, moving on to t

Model is infeasible, moving on to the next model...
Testing permutation ('B', 'C', 'D', 'A') at 9 intervals at 6.5 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('B', 'C', 'D', 'A') at 10 intervals at 6.5 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('B', 'C', 'D', 'A') at 11 intervals at 6.5 (s) half lap time.
Team work depletion for 11 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('B', 'D', 'A', 'C') at 7 intervals at 6.5 (s) half lap time.
Team work depletion for 7 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('B', 'D', 'A', 'C') at 8 intervals at 6.5 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('B', 'D', 'A', 'C') at 9 intervals at 6.5 (s) half la

Model is infeasible, moving on to the next model...
Testing permutation ('D', 'A', 'B', 'C') at 11 intervals at 6.5 (s) half lap time.
Team work depletion for 11 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'A', 'C', 'B') at 7 intervals at 6.5 (s) half lap time.
Team work depletion for 7 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'A', 'C', 'B') at 8 intervals at 6.5 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'A', 'C', 'B') at 9 intervals at 6.5 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'A', 'C', 'B') at 10 intervals at 6.5 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'A', 'C', 'B') at 11 intervals at 6.5 (s) half l

Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'B', 'C') at 8 intervals at 6.55 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'B', 'C') at 9 intervals at 6.55 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'B', 'C') at 10 intervals at 6.55 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'B', 'C') at 11 intervals at 6.55 (s) half lap time.
Team work depletion for 11 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'C', 'B') at 7 intervals at 6.55 (s) half lap time.
Team work depletion for 7 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'C', 'B') at 8 intervals at 6.55 (s) h

Model is infeasible, moving on to the next model...
Testing permutation ('C', 'A', 'D', 'B') at 10 intervals at 6.55 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'A', 'D', 'B') at 11 intervals at 6.55 (s) half lap time.
Team work depletion for 11 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'B', 'A', 'D') at 7 intervals at 6.55 (s) half lap time.
Team work depletion for 7 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'B', 'A', 'D') at 8 intervals at 6.55 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'B', 'A', 'D') at 9 intervals at 6.55 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'B', 'A', 'D') at 10 intervals at 6.55 (s) 

...

Testing permutation ('A', 'C', 'D', 'B') at 9 intervals at 6.65 (s) half lap time.
Team work depletion for 9 intervals: 107103.678031911
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'C', 'D', 'B') at 10 intervals at 6.65 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'C', 'D', 'B') at 11 intervals at 6.65 (s) half lap time.
Team work depletion for 11 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('A', 'D', 'B', 'C') at 7 intervals at 6.65 (s) half lap time.
Team work depletion for 7 intervals: -1
Testing permutation ('A', 'D', 'B', 'C') at 8 intervals at 6.65 (s) half lap time.
Team work depletion for 8 intervals: 106964.81314767673
Testing permutation ('A', 'D', 'B', 'C') at 9 intervals at 6.65 (s) half lap time.
Team work depletion for 9 intervals: 107250.41956147175
Model is infeasible, moving on to the next model...
Test

Model is infeasible, moving on to the next model...
Testing permutation ('C', 'A', 'D', 'B') at 8 intervals at 6.65 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'A', 'D', 'B') at 9 intervals at 6.65 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'A', 'D', 'B') at 10 intervals at 6.65 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'A', 'D', 'B') at 11 intervals at 6.65 (s) half lap time.
Team work depletion for 11 intervals: -1
Testing permutation ('C', 'B', 'A', 'D') at 7 intervals at 6.65 (s) half lap time.
Team work depletion for 7 intervals: 107550.98827449311
Model is infeasible, moving on to the next model...
Testing permutation ('C', 'B', 'A', 'D') at 8 intervals at 6.65 (s) half lap time.
Team work depletion fo

Testing permutation ('D', 'C', 'A', 'B') at 11 intervals at 6.65 (s) half lap time.
Team work depletion for 11 intervals: 107706.44492495744
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'C', 'B', 'A') at 7 intervals at 6.65 (s) half lap time.
Team work depletion for 7 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'C', 'B', 'A') at 8 intervals at 6.65 (s) half lap time.
Team work depletion for 8 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'C', 'B', 'A') at 9 intervals at 6.65 (s) half lap time.
Team work depletion for 9 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'C', 'B', 'A') at 10 intervals at 6.65 (s) half lap time.
Team work depletion for 10 intervals: -1
Model is infeasible, moving on to the next model...
Testing permutation ('D', 'C', 'B', 'A') at 11 intervals at 6.65 (s) half lap time.
Team work depletion f

Testing permutation ('B', 'D', 'A', 'C') at 9 intervals at 6.7 (s) half lap time.
Team work depletion for 9 intervals: 100627.48778695159
Testing permutation ('B', 'D', 'A', 'C') at 10 intervals at 6.7 (s) half lap time.
Team work depletion for 10 intervals: 100374.11821270114
Testing permutation ('B', 'D', 'A', 'C') at 11 intervals at 6.7 (s) half lap time.
Team work depletion for 11 intervals: 100657.10961026713
Testing permutation ('B', 'D', 'C', 'A') at 7 intervals at 6.7 (s) half lap time.
Team work depletion for 7 intervals: 100609.86682564582
Testing permutation ('B', 'D', 'C', 'A') at 8 intervals at 6.7 (s) half lap time.
Team work depletion for 8 intervals: 100492.21368404737
Testing permutation ('B', 'D', 'C', 'A') at 9 intervals at 6.7 (s) half lap time.
Team work depletion for 9 intervals: 100582.07455300828
Testing permutation ('B', 'D', 'C', 'A') at 10 intervals at 6.7 (s) half lap time.
Team work depletion for 10 intervals: 100870.45920999748
Testing permutation ('B', 'D

Testing permutation ('D', 'C', 'A', 'B') at 9 intervals at 6.7 (s) half lap time.
Team work depletion for 9 intervals: 101576.0825185464
Testing permutation ('D', 'C', 'A', 'B') at 10 intervals at 6.7 (s) half lap time.
Team work depletion for 10 intervals: 101657.08807753978
Testing permutation ('D', 'C', 'A', 'B') at 11 intervals at 6.7 (s) half lap time.
Team work depletion for 11 intervals: 101473.92780583946
Testing permutation ('D', 'C', 'B', 'A') at 7 intervals at 6.7 (s) half lap time.
Team work depletion for 7 intervals: 102084.11163723665
Testing permutation ('D', 'C', 'B', 'A') at 8 intervals at 6.7 (s) half lap time.
Team work depletion for 8 intervals: 101879.80553751897
Testing permutation ('D', 'C', 'B', 'A') at 9 intervals at 6.7 (s) half lap time.
Team work depletion for 9 intervals: 102248.65696141409
Testing permutation ('D', 'C', 'B', 'A') at 10 intervals at 6.7 (s) half lap time.
Team work depletion for 10 intervals: 102253.35503535799
Testing permutation ('D', 'C'